In [73]:
import oauth2 as oauth
import urllib2 as urllib
import json
import pandas as pd

# See assignment1.html instructions or README for how to get these credentials

api_key = "vT96WiMY9zGkQiaW1oxqRAWUp"
api_secret = "Hn2tEJrExaEPHbNq9uyZDhdeY9ro2Wx75lcimUPQ09lgVpSvC7"
access_token_key = "2723059178-LdJhYkZ7kV5qFrC1gAOs2vaRQOWiBA91JawaGQP"
access_token_secret = "pHjH9OPdN1O9LX0Aj1z6S0X9rKNRYm2vfheX9OmhSleB0"

_debug = 0

oauth_token    = oauth.Token(key=access_token_key, secret=access_token_secret)
oauth_consumer = oauth.Consumer(key=api_key, secret=api_secret)

signature_method_hmac_sha1 = oauth.SignatureMethod_HMAC_SHA1()

http_method = "GET"


http_handler  = urllib.HTTPHandler(debuglevel=_debug)
https_handler = urllib.HTTPSHandler(debuglevel=_debug)

'''
Construct, sign, and open a twitter request
using the hard-coded credentials above.
'''
def twitterreq(url, method, parameters):
  req = oauth.Request.from_consumer_and_token(oauth_consumer,
                                             token=oauth_token,
                                             http_method=http_method,
                                             http_url=url, 
                                             parameters=parameters)

  req.sign_request(signature_method_hmac_sha1, oauth_consumer, oauth_token)

  headers = req.to_header()

  if http_method == "POST":
    encoded_post_data = req.to_postdata()
  else:
    encoded_post_data = None
    url = req.to_url()

  opener = urllib.OpenerDirector()
  opener.add_handler(http_handler)
  opener.add_handler(https_handler)

  response = opener.open(url, encoded_post_data)

  return response



In [87]:
def fetchsamples():
  url = "https://api.twitter.com/1.1/search/tweets.json?count=150&lang=en&q=ParisAttacks&result_type=mixed"
  parameters = []
  response = twitterreq(url, "GET", parameters)
  statuses = []
  for line in response:
    line.strip()
    statuses.append(line)
  return statuses

In [88]:
tweet_feed = fetchsamples()

In [89]:
print len(tweet_feed)
parsed_feed = json.loads(tweet_feed[0])
print parsed_feed['statuses'][0]

1
{u'contributors': None, u'truncated': False, u'text': u'"They\'ve got guns but we have flowers."\nWatch a father\'s heartwarming talk with his young son about #ParisAttacks\nhttps://t.co/7o9xQIORAZ', u'is_quote_status': False, u'in_reply_to_status_id': None, u'id': 666686766959620096, u'favorite_count': 2321, u'source': u'<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', u'retweeted': False, u'coordinates': None, u'entities': {u'symbols': [], u'user_mentions': [], u'hashtags': [{u'indices': [100, 113], u'text': u'ParisAttacks'}], u'urls': [{u'url': u'https://t.co/7o9xQIORAZ', u'indices': [114, 137], u'expanded_url': u'https://amp.twimg.com/v/d2190d0d-1860-47ae-8008-6fe03f78670b', u'display_url': u'amp.twimg.com/v/d2190d0d-186\u2026'}]}, u'in_reply_to_screen_name': None, u'in_reply_to_user_id': None, u'retweet_count': 2117, u'id_str': u'666686766959620096', u'favorited': False, u'user': {u'follow_request_sent': False, u'has_extended_profile': False, u'profile_use_ba

In [90]:
print parsed_feed['statuses'][0].keys()
print '\n'
print parsed_feed['statuses'][0]['user'].keys()

[u'contributors', u'truncated', u'text', u'is_quote_status', u'in_reply_to_status_id', u'id', u'favorite_count', u'source', u'retweeted', u'coordinates', u'entities', u'in_reply_to_screen_name', u'in_reply_to_user_id', u'retweet_count', u'id_str', u'favorited', u'user', u'geo', u'in_reply_to_user_id_str', u'possibly_sensitive', u'lang', u'created_at', u'in_reply_to_status_id_str', u'place', u'metadata']


[u'follow_request_sent', u'has_extended_profile', u'profile_use_background_image', u'default_profile_image', u'id', u'profile_background_image_url_https', u'verified', u'profile_text_color', u'profile_image_url_https', u'profile_sidebar_fill_color', u'entities', u'followers_count', u'profile_sidebar_border_color', u'id_str', u'profile_background_color', u'listed_count', u'is_translation_enabled', u'utc_offset', u'statuses_count', u'description', u'friends_count', u'location', u'profile_link_color', u'profile_image_url', u'following', u'geo_enabled', u'profile_banner_url', u'profile_ba

In [94]:
# Variables needed to be extracted: location, created_at, text, user.screen_name, 
tweet_time = []
tweet_id = []
tweet_text = []
user_name = []
user_location = []
hashtags = []
print len(parsed_feed['statuses'])
tweets_list = parsed_feed['statuses']
for s in tweets_list:
    tweet_time_split = s['created_at'].split()
    tweet_h_split = tweet_time_split[3].split(':')
    tweet_hour = tweet_time_split[1]+' '+tweet_time_split[2]+' '+tweet_h_split[0]+' '+'hrs'
    tweet_time.append(tweet_hour)
    tweet_id.append(s['id'])
    tweet_text.append(s['text'])
    user_name.append(s['user']['screen_name'])
    user_location.append(s['user']['location'])
    hashtags_dict = s['entities']['hashtags']
    hashtags_list = [h['text'] for h in hashtags_dict]
    hashtags.append(hashtags_list)
    
twitter_df = pd.DataFrame({'time':tweet_time, 'id':tweet_id, 'text':tweet_text, 'hashtags':hashtags, 
                           'username':user_name, 'location':user_location})

twitter_df.head(5)

100


,hashtags,id,location,text,time,username
0,[ParisAttacks],666686766959620096,"New York, NY","""They've got guns but we have flowers.""\nWatch...",Nov 17 18 hrs,NBCNews
1,[Parisattacks],666695443296972800,,[#Parisattacks] French police wants to identif...,Nov 17 19 hrs,PNationale
2,"[ParisAttacks, Islamophobia]",666363017215410176,Moscow,"""Hug me if you trust me, I'm told I'm a terror...",Nov 16 21 hrs,INTHENOWRT
3,"[Texas, France, ParisAttacks]",667067353314795520,Brussels BE,RT @EUAmbUS: Touched by expression of support ...,Nov 18 19 hrs,AndyKlomEU
4,[ParisAttacks],667067352802983936,,RT @katiecouric: This video is so moving. A fa...,Nov 18 19 hrs,drpowersmd


In [105]:
%matplotlib inline
from bokeh.plotting import figure, show, output_file, vplot
time_coutns = twitter_df['time'].apply(pd.value_counts)

In [107]:
from collections import Counter
import numpy as np
# a = ['a', 'a', 'a', 'a', 'b', 'b', 'c', 'c', 'c', 'd', 'e', 'e', 'e', 'e', 'e']
letter_counts = Counter(twitter_df['time'].values)
df = pd.DataFrame.from_dict(letter_counts, orient='index')
# df.plot(kind='bar')
p1 = figure()
hist, edges = np.histogram(time_coutns, density=True, bins=50)

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
     fill_color="#036564", line_color="#033649")
show(p1)